In [32]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preliminary Data Cleaning
1. Load dataset in
2. Drop NA
3. Drop Outlier
4. Train test split
5. Scale the Age variable

In [33]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

df = pd.read_csv('lung cancer survey.csv')
df_no_na = df.dropna()

df_age = df_no_na[df_no_na["AGE"] > 21]
df_age

# Assuming 'age' is in the first column (index 0) of X
age_index = 1

X = df_age.drop('LUNG_CANCER', axis=1).values
y = df_age['LUNG_CANCER'].values
# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=888)

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler only on the age column in X_train
X_train_age = X_train[:, age_index].reshape(-1, 1)
scaler.fit(X_train_age)

# Transform the age column in X_train and X_val
X_train[:, age_index] = scaler.transform(X_train_age).flatten()
X_val[:, age_index] = scaler.transform(X_val[:, age_index].reshape(-1, 1)).flatten()

X_train

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,0.0,61.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,70.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2,1.0,59.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,54.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
4,0.0,54.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8996,1.0,62.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
8997,0.0,71.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
8998,1.0,63.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
8999,1.0,70.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0


StandardScaler()

array([[ 1.        ,  1.16977204,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 1.        , -1.2946817 ,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 0.        ,  0.64167481,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  1.25778824,  0.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        , -0.32650345,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 1.        , -1.11864929,  1.        , ...,  1.        ,
         0.        ,  1.        ]])

# Base Neural Network Model

The model below considers 3 hidden layers, each holding 10 nodes.

This is just a baseline model to understand the model parameters better

In [9]:
# Set seeds for reproducibility
random.seed(888)
tf.random.set_seed(888)
np.random.seed(888)

# Creating the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(15,)),  # Adjust input shape based on feature count
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
# Compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_val, y_val), verbose=1)


Epoch 1/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step - accuracy: 0.8286 - loss: 0.4472 - precision_1: 0.8263 - recall_1: 0.9983 - val_accuracy: 0.8578 - val_loss: 0.4038 - val_precision_1: 0.8828 - val_recall_1: 0.9469
Epoch 2/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 0s 550us/step - accuracy: 0.8861 - loss: 0.3721 - precision_1: 0.8861 - recall_1: 0.9860 - val_accuracy: 0.8744 - val_loss: 0.3680 - val_precision_1: 0.8821 - val_recall_1: 0.9721
Epoch 3/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step - accuracy: 0.8875 - loss: 0.3496 - precision_1: 0.8893 - recall_1: 0.9833 - val_accuracy: 0.8844 - val_loss: 0.3581 - val_precision_1: 0.8864 - val_recall_1: 0.9804
Epoch 4/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step - accuracy: 0.8935 - loss: 0.3360 - precision_1: 0.8918 - recall_1: 0.9882 - val_accuracy: 0.8883 - val_loss: 0.3455 - val_precision_1: 0.8788 - val_recall_1: 0.9972
Epoch 5/100
720/720 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - accuracy: 0.8940 - loss: 0.3307 - precision_1: 0.8929 - rec

In [10]:
# Get model predictions for validation set
y_val_pred = model.predict(X_val)
y_val_pred = (y_val_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate Precision, Recall, and F1-score
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step
Precision: 0.8782
Recall: 0.9972
F1-score: 0.9339


In [34]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense
from keras_tuner import HyperModel, RandomSearch, BayesianOptimization
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad

class MyHyperModel(HyperModel):
    def __init__(self, input_dim):
        self.input_dim = input_dim

    def build(self, hp):
        model = Sequential()
        model.add(layers.Input(shape=(self.input_dim,)))

        # First hidden layer with tunable units and activation
        model.add(Dense(
            units=hp.Int('units_1', min_value=8, max_value=45, step=3),
            activation=hp.Choice('activation_1', values=['relu', 'tanh', 'sigmoid', 'linear'])
        ))

        # Adding up to 5 more hidden layers based on the hp.Int range from 1 to 5 additional layers
        for i in range(hp.Int('num_layers', 1, 5)):  # This creates 2 to 6 total hidden layers
            model.add(Dense(
                units=hp.Int(f'units_{i+2}', min_value=8, max_value=45, step=3),
                activation=hp.Choice(f'activation_{i+2}', values=['relu', 'tanh', 'sigmoid', 'linear'])
            ))

        # Output layer for binary classification
        model.add(Dense(1, activation='sigmoid'))

        # Choosing the optimizer and learning rate as hyperparameters
        learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-1, sampling='log')
        optimizer_choice = hp.Choice('optimizer', values=['sgd', 'adam', 'rmsprop', 'adagrad'])

        # Define optimizer based on choice
        if optimizer_choice == 'sgd':
            opt = SGD(learning_rate=learning_rate)
        elif optimizer_choice == 'adam':
            opt = Adam(learning_rate=learning_rate)
        elif optimizer_choice == 'rmsprop':
            opt = RMSprop(learning_rate=learning_rate)
        elif optimizer_choice == 'adagrad':
            opt = Adagrad(learning_rate=learning_rate)
        else:
            raise ValueError("Optimizer not recognized.")

        # Compile model with selected optimizer
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
        
        return model


In [ ]:
# Assuming input_dim is the number of features (e.g., 15)
hypermodel = MyHyperModel(input_dim=15)

# Define the tuner
tuner = BayesianOptimization(
    hypermodel,
    objective='val_accuracy',
    max_trials= 1000,  # You can adjust the number of trials
    executions_per_trial=1,
    directory='my_dir',
    project_name='binary_classification'
)

# Start search
tuner.search(X_train, y_train, epochs=2000, batch_size=len(X_train), validation_data=(X_val, y_val))


Trial 36 Complete [00h 00m 43s]
val_accuracy: 0.7961111068725586

Best val_accuracy So Far: 0.9005555510520935
Total elapsed time: 00h 27m 19s

Search: Running Trial #37

Value             |Best Value So Far |Hyperparameter
20                |38                |units_1
tanh              |linear            |activation_1
5                 |5                 |num_layers
17                |23                |units_2
sigmoid           |linear            |activation_2
0.038902          |0.04038           |learning_rate
sgd               |adam              |optimizer
11                |8                 |units_3
tanh              |relu              |activation_3
17                |8                 |units_4
linear            |relu              |activation_4
20                |8                 |units_5
relu              |relu              |activation_5
17                |8                 |units_6
linear            |relu              |activation_6

Epoch 1/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319

In [14]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Get the best model from the tuner
best_model = tuner.get_best_models(num_models=1)[0]  # Gets the top model

# Make predictions on the validation set
y_val_pred = (best_model.predict(X_val) > 0.5).astype("int32")  # Threshold at 0.5 for binary classification

# Calculate F1 score
f1 = f1_score(y_val,y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)

print("Best model Precision on validation set:", precision)
print("Best model Recall on validation set:", recall)
print("Best model F1 score on validation set:", f1)




57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step
Best model Precision on validation set: 0.8881987577639752
Best model Recall on validation set: 0.9986033519553073
Best model F1 score on validation set: 0.9401709401709402
Best Hyperparameters: {'units_1': 26, 'activation_1': 'linear', 'num_layers': 5, 'units_2': 29, 'activation_2': 'relu', 'learning_rate': 0.0005543780243486915, 'optimizer': 'rmsprop', 'units_3': 44, 'activation_3': 'relu', 'units_4': 23, 'activation_4': 'sigmoid', 'units_5': 44, 'activation_5': 'sigmoid', 'units_6': 29, 'activation_6': 'tanh'}


/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# Set seeds for reproducibility
random.seed(888)
tf.random.set_seed(888)
np.random.seed(888)

learning_rate = 0.0005543780243486915
# Creating the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(15,)),  # Adjust input shape based on feature count
    tf.keras.layers.Dense(26, activation='linear'),
    tf.keras.layers.Dense(29, activation='relu'),
    tf.keras.layers.Dense(44, activation='relu'),
    tf.keras.layers.Dense(23, activation='sigmoid'),
    tf.keras.layers.Dense(44, activation='sigmoid'),
    tf.keras.layers.Dense(29, activation='tanh'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

opt = RMSprop(learning_rate=learning_rate)
# Compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1500, batch_size=len(X_train), verbose=1)

# Display optimized weights and biases after training (useful for analysis)
for layer in model.layers:
    weights, biases = layer.get_weights()
    print(f"Weights for {layer.name}:\n{weights}\nBiases:\n{biases}")

# Evaluate model on validation data (i.e., calculate loss function & accuracy performance metric of fitted model on validation data)
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

Epoch 1/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step - accuracy: 0.1926 - loss: 1.0743
Epoch 2/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1926 - loss: 0.9278
Epoch 3/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1926 - loss: 0.8392
Epoch 4/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1926 - loss: 0.7757
Epoch 5/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1926 - loss: 0.7268
Epoch 6/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7382 - loss: 0.6879
Epoch 7/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8074 - loss: 0.6562
Epoch 8/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8074 - loss: 0.6300
Epoch 9/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8074 - loss: 0.6080
Epoch 10/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8074 - loss: 0.5896
Epoch 11/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8074 - loss: 0.5739
Epoch 12/1500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy

Weights for dense_63:
[[-0.31983992 -0.22842342 -0.02145098 -0.02756065 -0.18586291  0.05300274
   0.33897752 -0.04884836  0.08385214  0.01313315 -0.50153184 -0.18484795
   0.43359646 -0.4508332  -0.14090607  0.21910317 -0.0391132   0.33721942
   0.01640136  0.02507787  0.4232161  -0.13054167  0.1305675  -0.01512093
  -0.20946403 -0.17992957]
 [ 0.19754957 -0.09249353 -0.17531736 -0.20746255  0.1950768   0.10957675
   0.06256054 -0.14943466 -0.31220925  0.00072567  0.20739949  0.4844266
   0.19191106 -0.21249753  0.28237507  0.12926896 -0.09149067  0.22302324
   0.16975279  0.08470135  0.14791742  0.1955754  -0.01743663  0.07877599
  -0.06438506  0.28235698]
 [ 0.16043669 -0.03367766 -0.04599079 -0.0269611  -0.22768928  0.5093278
  -0.08728761  0.01451697  0.34376457  0.16954197  0.2782854  -0.29208767
  -0.3154797   0.20724572 -0.36668968 -0.27075425  0.0726093   0.06744654
  -0.02517648 -0.04121419 -0.37585452 -0.02461718  0.15087487 -0.3763703
   0.26588148 -0.14644995]
 [ 0.4942495

In [31]:
# Get model predictions for validation set
y_val_pred = model.predict(X_val)
y_val_pred = (y_val_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate Precision, Recall, and F1-score
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 382us/step
Precision: 0.8877
Recall: 0.9937
F1-score: 0.9377
